In [77]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
86,tt3850590,15000000,61548707,Krampus,Adam Scott|Toni Collette|Allison Tolman|David ...,Michael Dougherty,You don't want to be on his list.,A horror comedy based on the ancient legend ab...,98,Horror|Comedy|Fantasy,Universal Pictures|Legendary Pictures,11/26/2015,5.9,2015
548,tt0206314,23000000,36642838,Joy Ride,Steve Zahn|Paul Walker|Leelee Sobieski|Jessica...,John Dahl,"It was just a joke, just for fun.",Three young people on a road trip from Colorad...,97,Mystery|Thriller|Drama,Regency Enterprises|Epsilon Motion Pictures|Li...,10/5/2001,6.2,2001
1485,tt0443543,40000000,86915017,The Illusionist,Edward Norton|Jessica Biel|Paul Giamatti|Rufus...,Neil Burger,Nothing Is What It Seems.,"With his eye on a lovely aristocrat, a gifted ...",110,Fantasy|Drama|Thriller|Romance,Michael London Productions|Stillking Films|Bob...,8/18/2006,7.0,2006
1229,tt2193215,25000000,71009334,The Counselor,Michael Fassbender|Cameron Diaz|Javier Bardem|...,Ridley Scott,Sin Is A Choice.,A rich and successful lawyer named Counselor i...,117,Thriller|Crime|Drama,Ingenious Media|Fox 2000 Pictures|Scott Free P...,10/25/2013,4.9,2013
1239,tt3063516,15000000,151831537,Jackass Presents: Bad Grandpa,Johnny Knoxville|Jackson Nicoll|Georgina Cates...,Jeff Tremaine,Real people. Real reactions. Real messed up.,86-year-old Irving Zisman is on a journey acro...,92,Comedy,Paramount Pictures|MTV Films|Dickhouse Product...,10/22/2013,6.1,2013


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [73]:
answers = {} # создадим словарь для ответов


data['release_date'] = pd.to_datetime(data['release_date']) # convert release_date from object to datetime64

data['profit'] = data['revenue'] - data['budget'] # create new column for profit values

data['month'] = data.release_date.dt.month # create new column for month

data['companies_list'] = data.production_companies.str.split("|") # for question 24

data['title_length'] = data.original_title.apply(lambda x: len(x)) # for question 24

data['title_word_length'] = data.overview.apply(lambda x: len(x)) # for question 25

data.head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,1363528810,6,"[Universal Studios, Amblin Entertainment, Lege...",14,175
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,228436354,5,"[Village Roadshow Pictures, Kennedy Miller Pro...",18,581
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,185238201,3,"[Summit Entertainment, Mandeville Films, Red W...",9,139


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [8]:
# в словарь вставляем номер вопроса и ваш ответ на него

answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
#+

In [9]:
data.groupby(['budget'])[['original_title','imdb_id']].max().sort_values(['budget'], ascending=False).head(1)


,original_title,imdb_id
budget,,
380000000,Pirates of the Caribbean: On Stranger Tides,tt1298650


Variation 2

In [10]:
data[data.budget==data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,641683000,5,"[Walt Disney Pictures, Jerry Bruckheimer Films...",43,411


# 2. Какой из фильмов самый длительный (в минутах)?

In [11]:
answers['2'] = '1157. Gods and Generals (tt0279111)'
#+

In [12]:
data.groupby(['runtime'])[['original_title','imdb_id']].max().sort_values(['runtime'], ascending=False).head(1)

,original_title,imdb_id
runtime,,
214,Gods and Generals,tt0279111


Variation 2

In [13]:
data[data.runtime==data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2003-02-21,5.8,2003,-43076064,2,"[Turner Pictures, Antietam Filmworks]",17,307


# 3. Какой из фильмов самый короткий (в минутах)?





In [14]:
answers['3'] = '768. Winnie the Pooh (tt1449283)'
#+

In [15]:
data.groupby(['runtime'])[['original_title','imdb_id']].min().sort_values(['runtime'], ascending=True).head(1)

,original_title,imdb_id
runtime,,
63,Winnie the Pooh,tt1449283


Variation 2

In [16]:
data[data.runtime==data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,2011-04-13,6.8,2011,-15540000,4,"[Walt Disney Pictures, Walt Disney Animation S...",15,305


# 4. Какова средняя длительность фильмов?


In [17]:
answers['4'] = '110'
#+

In [18]:
result = data.runtime.mean()
round(result)

110

# 5. Каково медианное значение длительности фильмов? 

In [19]:
answers['5'] = '107'
#+

In [20]:
result2 = data.runtime.median()
round(result2)

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [21]:
answers['6'] = '239. Avatar (tt0499549)'
#+

In [22]:
data.groupby(['profit'])[['original_title','imdb_id']].max().sort_values(['profit'], ascending=False).head(1)

,original_title,imdb_id
profit,,
2544505847,Avatar,tt0499549


Variation 2

In [23]:
data[data.profit==data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,2009-12-10,7.1,2009,2544505847,12,"[Ingenious Film Partners, Twentieth Century Fo...",6,175


# 7. Какой фильм самый убыточный? 

In [24]:
answers['7'] = '1245. The Lone Ranger (tt1210819)'
#+

In [25]:
data.groupby(['profit'])[['original_title','imdb_id']].min().sort_values(['profit'], ascending=True).head(1)

,original_title,imdb_id
profit,,
-165710090,The Lone Ranger,tt1210819


Variation 2

In [26]:
data[data.profit==data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090,7,"[Walt Disney Pictures, Jerry Bruckheimer Films...",15,386


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [27]:
answers['8'] = '1478'
#+

In [28]:
len(data[data.profit>0])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [29]:
answers['9'] = '599. The Dark Knight (tt0468569)'
#+

In [30]:
film_2008 = data[data['release_year']==2008]
film_2008[film_2008.revenue==film_2008.revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008,816921825,7,"[DC Comics, Legendary Pictures, Warner Bros., ...",15,396


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [31]:
answers['10'] = '1245. The Lone Ranger (tt1210819)'
#+

In [32]:
film_det = data.query('2012<=release_year<=2014')
film_det[film_det.profit==film_det.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090,7,"[Walt Disney Pictures, Jerry Bruckheimer Films...",15,386


# 11. Какого жанра фильмов больше всего?

In [33]:
answers['11'] = 'Drama'
#

In [34]:
'''Counter - вид словаря,
который считает количество неизменяемых объектов'''

'''метод most_common() возвращает n наиболее часто 
встречающихся элементов, в порядке убывания встречаемости'''

genre_list = []
for line in data.genres:
    genre_list.extend(line.split('|')) 
Counter(genre_list).most_common(1)

[('Drama', 782)]

ВАРИАНТ 2

In [35]:
genre_max = Counter()
for x in data['genres'].str.split('|'):
    genre_max += Counter(x)
genre_max.most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [36]:
answers['12'] = 'Drama'
#+

In [37]:
data_only_profit = data[data.profit>0]

genre_list = []
for line in data_only_profit.genres:
    genre_list.extend(line.split('|'))
Counter(genre_list).most_common(1)          


[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [38]:
answers['13'] = 'Peter Jackson'
#+

In [39]:
data.groupby(['director'])['revenue'].sum().sort_values(ascending=False).index[0]


'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [40]:
answers['14'] = 'Robert Rodriguez'
#+

In [41]:
data_action_director = data[data.genres.str.contains('Action')]
director_list = []
for line in data_action_director.director:
    director_list.extend(line.split('|')) 
Counter(director_list).most_common(1)

[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [42]:
answers['15'] = 'Chris Hemsworth'
#+

In [43]:
data_2012 = data.query('release_year==2012')

count = Counter()
for i in range(0, len(data_2012)):
    for k in data_2012.iloc[i]['cast'].split('|'):
        count[k] += data_2012.iloc[i]['revenue']
count.most_common(1)

[('Chris Hemsworth', 2027450773)]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [44]:
answers['16'] = 'Matt Damon'
#+

In [45]:
data_high = data.query('budget>budget.mean()') # budget value should be higher than mean value of budget value
actor_list = []
for line in data_high.cast:
    actor_list.extend(line.split('|')) 
Counter(actor_list).most_common(1)

[('Matt Damon', 18)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [46]:
answers['17'] = 'Action'
#+

In [47]:
data_cage = data[data.cast.str.contains('Nicolas Cage',na=False)]
genre_list = []
for line in data_cage.genres:
    genre_list.extend(line.split('|'))
Counter(genre_list).most_common(1)

[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [48]:
answers['18'] = '925. K-19: The Widowmaker (tt0267626)'
#+

In [49]:
data_paramount = data[data.production_companies.str.contains('Paramount Pictures',na=False)]
data_paramount[data_paramount.profit==data_paramount.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,companies_list,title_length,title_word_length
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,2002-07-19,6.0,2002,-64831034,7,"[Paramount Pictures, Intermedia Films, Nationa...",20,140


Variation 2

In [50]:
data_paramount = data[data.production_companies.str.contains('Paramount Pictures', na=False)]
min_paramount = data_paramount.loc[data_paramount['profit'] == data_paramount['profit'].min()]
display(min_paramount[['original_title', 'imdb_id']])

,original_title,imdb_id
925,K-19: The Widowmaker,tt0267626


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [51]:
answers['19'] = '2015'
#+

In [52]:
df_year_max = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)
display(df_year_max.index[0])

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [53]:
answers['20'] = '2014'
#+

In [54]:
df_warner = data[data.production_companies.str.contains('Warner Bros', na=False)]
df_warner = df_warner.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)
display(df_warner.index[0])

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [55]:
answers['21'] = 'Сентябрь'
#+

In [56]:
df_month = data.groupby(['month'])['imdb_id'].count().sort_values(ascending=False).head(1)
display(df_month.index[0])

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [57]:
answers['22'] = '450'
#+

In [58]:
data_summer = data.query('month=="6" or month=="7" or month=="8"')
data_summer.imdb_id.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [59]:
answers['23'] = 'Peter Jackson'
#+

In [60]:
data_winter = data.query('month=="12" or month=="1" or month=="2"')
director_list = []
for line in data_winter.director:
    director_list.extend(line.split('|')) 
Counter(director_list).most_common(1)

[('Peter Jackson', 7)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [61]:
answers['24'] = 'Four By Two Productions'
#+

In [62]:
data_exploded = data.explode('companies_list') # each movie is repeated for each company
data_exploded = data_exploded.groupby(['companies_list'])['title_length'].mean().sort_values(ascending=False).head(1)
display(data_exploded.index[0])

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [63]:
answers['25'] = 'Midnight Picture Show'
#+

In [64]:
data_exploded = data.explode('companies_list') 
data_exploded = data_exploded.groupby(['companies_list'])['title_word_length'].mean().sort_values(ascending=False).head(1)
display(data_exploded.index[0])

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [65]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
#+

In [66]:
data_top1 = data[data['vote_average'] > data.quantile(0.99, numeric_only=True)['vote_average']]
data_top1[['original_title', 'vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [67]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
#+

In [81]:
pair_list = []
for line in data.cast:
    current_line = combinations(line.split('|'), 2)
    pair_list.extend(current_line)
Counter(pair_list).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

Variation 2

In [86]:
data_actor = data.cast.str.split('|').tolist()
list_actors = []
for name in data_actor:
    for variantion in combinations(name, 2):
        list_actors.append(' '.join(variantion))
list_actors = pd.DataFrame(list_actors)
list_actors.columns = ['actor_together']
list_actors.actor_together.value_counts().head(1)

Daniel Radcliffe Emma Watson    8
Name: actor_together, dtype: int64

# Submission

In [147]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': '925. K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [148]:
# и убедиться что ни чего не пропустил)
len(answers)

27